In [1]:
import pandas as pd

df = pd.read_csv('ORDFlights2008.csv')

In [3]:
df.head().T

,0,1,2,3,4
Month,1,1,1,1,1
DayofMonth,8,25,10,4,22
DayOfWeek,2,5,4,5,2
DepTime,1711,1813,1601,658,725
CRSDepTime,1600,1742,1600,650,725
ArrTime,2031,2040,1928,1002,834
CRSArrTime,1945,2004,1945,955,757
UniqueCarrier,XE,XE,XE,XE,XE
FlightNum,1226,2229,1226,1220,2408
TailNum,N14953,N15509,N14974,N14974,N11119


In [2]:
orgOrdDf=df[df['Origin']=='ORD']

In [4]:
destOrdDf=df[df['Dest']=='ORD']

In [14]:
destOrdDf.head().T

,4,6,12,13,14
Month,1,1,1,1,1
DayofMonth,22,4,8,25,9
DayOfWeek,2,5,2,5,3
DepTime,725,1038,1841,1100,628
CRSDepTime,725,1040,1635,1040,630
ArrTime,834,1158,1920,1220,759
CRSArrTime,757,1210,1712,1210,759
UniqueCarrier,XE,XE,XE,XE,XE
FlightNum,2408,1221,2505,1221,1231
TailNum,N11119,N14974,N13992,N17159,N14543


In [5]:
delayedFlights = orgOrdDf[orgOrdDf['DepDelay']>30.0].groupby(['DepHr', 'DayofMonth', 'Month']).count().add_suffix('_del_count')
delayedFlights=delayedFlights[['Dest_del_count']].reset_index()
allFlights = orgOrdDf.groupby(['DepHr', 'DayofMonth', 'Month']).count().add_suffix('_all_count')
allFlights=allFlights[['Dest_all_count']].reset_index()
jFlightInfo =pd.merge(delayedFlights,allFlights,on=['DepHr','DayofMonth','Month'])
jFlightInfo['percent_delay']=jFlightInfo['Dest_del_count'].div(jFlightInfo['Dest_all_count'])
jFlightInfo.drop(['Dest_del_count','Dest_all_count'], axis=1, inplace=True)
interim=pd.merge(orgOrdDf,allFlights,on=['DepHr','DayofMonth','Month'])
dfWithPercentDelay=pd.merge(interim,jFlightInfo,on=['DepHr','DayofMonth','Month'],how='left')

In [88]:
dfWithPercentDelay.head().T

,0,1,2,3,4
Month,1,1,1,1,1
DayofMonth,8,8,8,8,8
DayOfWeek,2,2,2,2,2
DepTime,1711,1926,1919,1920,1615
CRSDepTime,1600,1645,1625,1625,1615
ArrTime,2031,2205,2217,2025,1846
CRSArrTime,1945,1934,1920,1723,1839
UniqueCarrier,XE,YV,YV,YV,OH
FlightNum,1226,7297,7230,7155,5094
TailNum,N14953,N511MJ,N715SF,N592ML,N430CA


In [6]:
delayedFlights = orgOrdDf[orgOrdDf['DepDelay']>30.0].groupby(['DepHr', 'DayofMonth', 'Month','UniqueCarrier']).count().add_suffix('_del_per_carr_count')
delayedFlights=delayedFlights[['Dest_del_per_carr_count']].reset_index()
allFlights = orgOrdDf.groupby(['DepHr', 'DayofMonth', 'Month','UniqueCarrier']).count().add_suffix('_per_carr_count')
allFlights=allFlights[['Dest_per_carr_count']].reset_index()
jFlightInfo =pd.merge(delayedFlights,allFlights,on=['DepHr','DayofMonth','Month','UniqueCarrier'])
jFlightInfo['per_del_per_carr']=jFlightInfo['Dest_del_per_carr_count'].div(jFlightInfo['Dest_per_carr_count'])
jFlightInfo.drop(['Dest_del_per_carr_count','Dest_per_carr_count'], axis=1, inplace=True)
interim=pd.merge(dfWithPercentDelay,allFlights,on=['DepHr','DayofMonth','Month','UniqueCarrier'])
dfWithPercentDelayPerAircraft=pd.merge(interim,jFlightInfo,on=['DepHr','DayofMonth','Month','UniqueCarrier'],how='left')
dfWithPercentDelayPerAircraft=dfWithPercentDelayPerAircraft.fillna(value=0.0)

In [7]:
interim = pd.merge(destOrdDf,orgOrdDf,on=['DayofMonth','Month','UniqueCarrier','TailNum'])
print interim.shape
res= interim[(interim['ArrHr_x']-interim['DepHr_y'] <= 0) & (interim['ArrHr_x']-interim['DepHr_y'] > -5)]

(737740, 42)


In [8]:
#res[['DayofMonth','Month','UniqueCarrier','TailNum','ArrHr_x','DepHr_y']]
#ArrTime_all_Flight
linkedFlght = res.groupby(['UniqueCarrier','TailNum']).count().add_suffix('_linked_Flight')
linkedFlght=linkedFlght[['ArrTime_x_linked_Flight']].reset_index()
allIncomFl = destOrdDf.groupby(['UniqueCarrier','TailNum']).count().add_suffix('_all_Flight')
allIncomFl=allIncomFl[['ArrTime_all_Flight']].reset_index()
linkinFlInfo=pd.merge(linkedFlght,allIncomFl, on=['UniqueCarrier','TailNum'])
linkinFlInfo['per_linkin_per_carr']=linkinFlInfo['ArrTime_x_linked_Flight'].div(linkinFlInfo['ArrTime_all_Flight'])
linkinFlInfo.drop(['ArrTime_x_linked_Flight','ArrTime_all_Flight'], axis=1, inplace=True)
linkinFlInfo=linkinFlInfo[linkinFlInfo['per_linkin_per_carr'] > .70]
dfWithlinkTailNum=pd.merge(dfWithPercentDelayPerAircraft,linkinFlInfo,on=['UniqueCarrier','TailNum'],how='left')
dfWithlinkTailNum=dfWithlinkTailNum.fillna(value=0.0)

In [9]:
indexed_destOrdDf=destOrdDf.set_index(['Month','DayofMonth','DepHr','UniqueCarrier','TailNum'])

In [10]:
count =0
Rows=[]
for index,row in dfWithlinkTailNum.iterrows():
    #print row
    count=count+1
    if count%10000==0:
        print count
    
    #print row
    
    if(row['per_linkin_per_carr']>=.7):
        #row2 = indexed_destOrdDf[(indexed_destOrdDf['Month']==row['Month']) & (indexed_destOrdDf['DayofMonth']==row['DayofMonth'])  & (indexed_destOrdDf['DepHr']<=row['DepHr']) & (destOrdDf['UniqueCarrier']==row['UniqueCarrier']) & (indexed_destOrdDf['TailNum']==row['TailNum'])]
        
        row2 = indexed_destOrdDf.loc[(indexed_destOrdDf.index.get_level_values('Month')==row['Month']) & (indexed_destOrdDf.index.get_level_values('DayofMonth')==row['DayofMonth'])  & (indexed_destOrdDf.index.get_level_values('DepHr')<=row['DepHr']) & (indexed_destOrdDf.index.get_level_values('UniqueCarrier')==row['UniqueCarrier']) & (indexed_destOrdDf.index.get_level_values('TailNum')==row['TailNum'])]
        
        #print row2.shape
        
        if row2.shape[0]==0:
            continue
        
        tempRow=None
        for index3,row3 in row2.iterrows():
            tempRow=row3
        
        
        row2=tempRow
        
        
        
        if( row2['DepDelay']<=0):
            row['IncomingDelay']=0.0
        elif(0< row2['DepDelay'] and  row2['DepDelay'] <=15):
            row['IncomingDelay']=1.0
        elif(15< row2['DepDelay'] and  row2['DepDelay'] <=30):
            row['IncomingDelay']=2.0
        else:
            row['IncomingDelay']=3.0
            
    else:
        row['IncomingDelay']=0.0
        
    Rows.append(row)
        
    #print row
    
    #print row2
    
    #break

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000


In [11]:
toBediscretizedDf = pd.DataFrame(Rows)
toBediscretizedDf = toBediscretizedDf

In [12]:
dfWithlinkTailNum[dfWithlinkTailNum['UniqueCarrier']=='9E']

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,TaxiIn,TaxiOut,Diverted,DepHr,ArrHr,Dest_all_count,percent_delay,Dest_per_carr_count,per_del_per_carr,per_linkin_per_carr
309117,11,29,6,556,600,818.0,816,9E,5854,96069E,...,5.0,34,0,6,8,41,0.000000,1,0.0,0.75
313849,12,16,2,924,925,1233.0,1225,9E,5800,91669E,...,9.0,28,0,9,12,65,0.169231,1,0.0,1.00
326574,12,16,2,1506,1450,1837.0,1756,9E,5808,91539E,...,7.0,54,0,14,17,37,0.513514,1,0.0,1.00
326749,12,16,2,2240,1855,231.0,2155,9E,5840,91629E,...,11.0,82,0,18,21,43,1.000000,1,1.0,1.00
327986,12,18,4,1402,1347,1612.0,1610,9E,5824,91479E,...,10.0,17,0,13,16,66,0.136364,1,0.0,1.00
328564,12,19,5,1409,1347,1642.0,1610,9E,5824,91479E,...,8.0,39,0,13,16,56,0.589286,1,0.0,1.00
329253,12,20,6,1353,1347,1614.0,1610,9E,5824,96009E,...,10.0,24,0,13,16,53,0.301887,1,0.0,1.00
329713,12,21,7,1408,1347,1618.0,1610,9E,5824,91769E,...,4.0,23,0,13,16,53,0.660377,1,0.0,1.00
330324,12,22,1,1356,1347,1620.0,1610,9E,5824,96009E,...,10.0,18,0,13,16,61,0.409836,1,0.0,1.00
331193,12,24,3,1505,1340,1711.0,1603,9E,5824,96069E,...,6.0,13,0,13,16,54,0.722222,1,1.0,0.75


In [39]:
toBediscretizedDf[toBediscretizedDf['UniqueCarrier']=='9E']

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,...,TaxiOut,Diverted,DepHr,ArrHr,Dest_all_count,percent_delay,Dest_per_carr_count,per_del_per_carr,per_linkin_per_carr,IncomingDelay
313849,12,16,2,924,925,1233.0,1225,9E,5800,91669E,...,28,0,9,12,65,0.169231,1,0.0,1.00,0.0
326574,12,16,2,1506,1450,1837.0,1756,9E,5808,91539E,...,54,0,14,17,37,0.513514,1,0.0,1.00,1.0
326749,12,16,2,2240,1855,231.0,2155,9E,5840,91629E,...,82,0,18,21,43,1.000000,1,1.0,1.00,3.0
327986,12,18,4,1402,1347,1612.0,1610,9E,5824,91479E,...,17,0,13,16,66,0.136364,1,0.0,1.00,1.0
328564,12,19,5,1409,1347,1642.0,1610,9E,5824,91479E,...,39,0,13,16,56,0.589286,1,0.0,1.00,1.0
329253,12,20,6,1353,1347,1614.0,1610,9E,5824,96009E,...,24,0,13,16,53,0.301887,1,0.0,1.00,0.0
329713,12,21,7,1408,1347,1618.0,1610,9E,5824,91769E,...,23,0,13,16,53,0.660377,1,0.0,1.00,1.0
330324,12,22,1,1356,1347,1620.0,1610,9E,5824,96009E,...,18,0,13,16,61,0.409836,1,0.0,1.00,0.0
331193,12,24,3,1505,1340,1711.0,1603,9E,5824,96069E,...,13,0,13,16,54,0.722222,1,1.0,0.75,3.0
331658,12,25,4,1340,1340,1555.0,1603,9E,5824,96029E,...,22,0,13,16,62,0.274194,1,0.0,1.00,1.0


In [36]:
dfWithlinkTailNum.groupby('UniqueCarrier').count()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,TailNum,ActualElapsedTime,...,TaxiIn,TaxiOut,Diverted,DepHr,ArrHr,Dest_all_count,percent_delay,Dest_per_carr_count,per_del_per_carr,per_linkin_per_carr
UniqueCarrier,,,,,,,,,,,,,,,,,,,,,
9E,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18
AA,67624,67624,67624,67624,67624,67624,67624,67624,67624,67624,...,67624,67624,67624,67624,67624,67624,67624,67624,67624,67624
AS,1440,1440,1440,1440,1440,1440,1440,1440,1440,1440,...,1440,1440,1440,1440,1440,1440,1440,1440,1440,1440
B6,2288,2288,2288,2288,2288,2288,2288,2288,2288,2288,...,2288,2288,2288,2288,2288,2288,2288,2288,2288,2288
CO,4888,4888,4888,4888,4888,4888,4888,4888,4888,4888,...,4888,4888,4888,4888,4888,4888,4888,4888,4888,4888
DL,3471,3471,3471,3471,3471,3471,3471,3471,3471,3471,...,3471,3471,3471,3471,3471,3471,3471,3471,3471,3471
EV,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
MQ,83111,83111,83111,83111,83111,83111,83111,83111,83111,83111,...,83111,83111,83111,83111,83111,83111,83111,83111,83111,83111
NW,6562,6562,6562,6562,6562,6562,6562,6562,6562,6562,...,6562,6562,6562,6562,6562,6562,6562,6562,6562,6562


In [37]:
toBediscretizedDf.groupby('UniqueCarrier').count()

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,TailNum,ActualElapsedTime,...,TaxiOut,Diverted,DepHr,ArrHr,Dest_all_count,percent_delay,Dest_per_carr_count,per_del_per_carr,per_linkin_per_carr,IncomingDelay
UniqueCarrier,,,,,,,,,,,,,,,,,,,,,
9E,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
AA,55016,55016,55016,55016,55016,55016,55016,55016,55016,55016,...,55016,55016,55016,55016,55016,55016,55016,55016,55016,55016
AS,1403,1403,1403,1403,1403,1403,1403,1403,1403,1403,...,1403,1403,1403,1403,1403,1403,1403,1403,1403,1403
B6,1913,1913,1913,1913,1913,1913,1913,1913,1913,1913,...,1913,1913,1913,1913,1913,1913,1913,1913,1913,1913
CO,4285,4285,4285,4285,4285,4285,4285,4285,4285,4285,...,4285,4285,4285,4285,4285,4285,4285,4285,4285,4285
DL,3106,3106,3106,3106,3106,3106,3106,3106,3106,3106,...,3106,3106,3106,3106,3106,3106,3106,3106,3106,3106
EV,23,23,23,23,23,23,23,23,23,23,...,23,23,23,23,23,23,23,23,23,23
MQ,77841,77841,77841,77841,77841,77841,77841,77841,77841,77841,...,77841,77841,77841,77841,77841,77841,77841,77841,77841,77841
NW,6065,6065,6065,6065,6065,6065,6065,6065,6065,6065,...,6065,6065,6065,6065,6065,6065,6065,6065,6065,6065


In [14]:
discretizedDf = pd.DataFrame(Rows)
discretizedDf = discretizedDf.T

In [15]:
sortedToBediscretizedDf=toBediscretizedDf.sort(columns=['Month','DayofMonth','DepHr'])

E:\Software\AnacondaForPython\installation\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [16]:
def determinePrevHr(mon,dayOfMon,hr):
    monToLastDateMap={1:31, 2:29, 3:31, 4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    
    if(hr==0):
        if(dayOfMon==1):
            if(mon==1):
                return 12, monToLastDateMap[mon-1], 24 
            else:    
                return mon-1, monToLastDateMap[mon-1], 24 
        else:
            return mon, dayOfMon-1, 24    
    else:
        return mon, dayOfMon,hr-1

In [17]:

monDayHrDelayInfo={}

for index,row in dfWithlinkTailNum.iterrows():
    mon= row['Month']
    dayOfMon = row['DayofMonth']
    hr = row['DepHr']
    
    if mon not in monDayHrDelayInfo:
        monDayHrDelayInfo[mon]={}
    
    if dayOfMon not in monDayHrDelayInfo[mon]:
        monDayHrDelayInfo[mon][dayOfMon]={}
    
    if hr not in monDayHrDelayInfo[mon][dayOfMon]:
        monDayHrDelayInfo[mon][dayOfMon][hr]={}
        monDayHrDelayInfo[mon][dayOfMon][hr]['Dest_all_count']=row['Dest_all_count']
        monDayHrDelayInfo[mon][dayOfMon][hr]['percent_delay']=row['percent_delay']
        monDayHrDelayInfo[mon][dayOfMon][hr]['Dest_per_carr_count']=row['Dest_per_carr_count']
        monDayHrDelayInfo[mon][dayOfMon][hr]['per_del_per_carr']=row['per_del_per_carr']
    
#for index,row in sortedToBediscretizedDf.iterrows():
    

In [18]:
count=-1
RowsWithPrevHrInfo={}
for index,row in sortedToBediscretizedDf.iterrows():
    count+=1
    prevMon,prevDayOfMon,prevHr = determinePrevHr(row['Month'], row['DayofMonth'], row['DepHr'])
    
    try:
        row['prevDest_all_count']=monDayHrDelayInfo[prevMon][prevDayOfMon][prevHr]['Dest_all_count']
        row['prevpercent_delay']=monDayHrDelayInfo[prevMon][prevDayOfMon][prevHr]['percent_delay']
        row['prevDest_per_carr_count']=monDayHrDelayInfo[prevMon][prevDayOfMon][prevHr]['Dest_per_carr_count']
        row['prevper_del_per_carr']=monDayHrDelayInfo[prevMon][prevDayOfMon][prevHr]['per_del_per_carr']
    except KeyError, e:
        row['prevDest_all_count']=0
        row['prevpercent_delay']=0
        row['prevDest_per_carr_count']=0
        row['prevper_del_per_carr']=0
        
    
    RowsWithPrevHrInfo[count]=row


In [19]:
toBeDisWithPrevHrDf = pd.DataFrame(RowsWithPrevHrInfo)
toBeDisWithPrevHrDf = toBeDisWithPrevHrDf.T

In [1]:
toBeDisWithPrevHrDf.head().T

NameError: name 'toBeDisWithPrevHrDf' is not defined

In [20]:
count =-1
Rows1={}
carrToCode={'CO':0, 'XE':1, 'OO':2, 'MQ':3, 'NW':4, 'AA':5, 'US':6, 'DL':7, 'YV':8, 'UA':9, 'AS':10,
       'B6':11, 'OH':12, 'EV':13, '9E':14}
for index1,row1 in toBeDisWithPrevHrDf.iterrows():
    count+=1
    row={}
    row['Month']=row1['Month']
    
    if row1['DayofMonth']<=3:
        row['DayofMonth']=0.0
    elif 3<row1['DayofMonth'] and row1['DayofMonth']<=27:
        row['DayofMonth']=1.0
    else:
        row['DayofMonth']=2.0
        
    if row1['DayOfWeek']<=1:
        row['DayOfWeek']=2.0
    elif 2<=row1['DayOfWeek'] and row1['DayOfWeek']<=4:
        row['DayOfWeek']=0.0
    elif row1['DayOfWeek']==5:
        row['DayOfWeek']=3.0
    else:
        row1['DayOfWeek']=1.0
        
    row['ActualDepTime'] = row1['DepTime']//100.0
    
    row['ActualArrTime'] = row1['ArrTime']//100.0
    
    row['UniqueCarrier'] = carrToCode[row1['UniqueCarrier']]
    
    if row1['ArrDelay']>=15:
        row['ArrDelay']=1.0
    else:
        row['ArrDelay']=0.0
        
    if row1['DepDelay']>=15:
        row['DepDelay']=1.0
    else:
        row['DepDelay']=0.0
        
        
    if 0<=row1['Distance'] and row1['Distance']<=200:
        row['Distance']=0.0
    elif 200<row1['Distance'] and row1['Distance']<=400:
        row['Distance']=1.0
    elif 400<row1['Distance'] and row1['Distance']<=600:
        row['Distance']=2.0
    elif 600<row1['Distance'] and row1['Distance']<=800:
        row['Distance']=3.0
    else:
        row['Distance']=4.0
        
    
    row['DepHr']=row1['DepHr']
    
    row['ArrHr']=row1['ArrHr']
    
    if 0<=row1['Dest_all_count'] and row1['Dest_all_count']<=5:
        row['ApSchDepCnt']=0.0
    elif 5<row1['Dest_all_count'] and row1['Dest_all_count']<=20:
        row['ApSchDepCnt']=1.0
    elif 20<row1['Dest_all_count'] and row1['Dest_all_count']<=50:
        row['ApSchDepCnt']=2.0
    elif 50<row1['Dest_all_count'] and row1['Dest_all_count']<=70:
        row['ApSchDepCnt']=3.0
    else:
        row['ApSchDepCnt']=4.0
        
        
    if 0<=row1['percent_delay'] and row1['percent_delay']<=0.1:
        row['ApPerDelay']=0.0
    elif 0.1<row1['percent_delay'] and row1['percent_delay']<=0.4:
        row['ApPerDelay']=1.0
    elif 0.4<row1['percent_delay'] and row1['percent_delay']<=0.7:
        row['ApPerDelay']=2.0
    else:
        row['ApPerDelay']=3.0
        
    if 0<=row1['Dest_per_carr_count'] and row1['Dest_per_carr_count']<=5:
        row['CarSchDepCnt']=0.0
    elif 5<row1['Dest_per_carr_count'] and row1['Dest_per_carr_count']<=10:
        row['CarSchDepCnt']=1.0
    elif 10<row1['Dest_per_carr_count'] and row1['Dest_per_carr_count']<=15:
        row['CarSchDepCnt']=2.0
    else:
        row['CarSchDepCnt']=3.0
        
        
    if 0<=row1['per_del_per_carr'] and row1['per_del_per_carr']<=0.1:
        row['CarPerDelay']=0.0
    elif 0.1<row1['per_del_per_carr'] and row1['per_del_per_carr']<=0.4:
        row['CarPerDelay']=1.0
    elif 0.4<row1['per_del_per_carr'] and row1['per_del_per_carr']<=0.7:
        row['CarPerDelay']=2.0
    else:
        row['CarPerDelay']=3.0
        
        
    if row1['per_linkin_per_carr']>=.7:
        row['linkedFlght']=1.0
    else:
        row['linkedFlght']=0.0
    
    row['IncomingDelay']=row1['IncomingDelay']
    
    if 0<=row1['prevDest_all_count'] and row1['prevDest_all_count']<=5:
        row['prevApSchDepCnt']=0.0
    elif 5<row1['prevDest_all_count'] and row1['prevDest_all_count']<=20:
        row['prevApSchDepCnt']=1.0
    elif 20<row1['prevDest_all_count'] and row1['prevDest_all_count']<=50:
        row['prevApSchDepCnt']=2.0
    elif 50<row1['prevDest_all_count'] and row1['prevDest_all_count']<=70:
        row['prevApSchDepCnt']=3.0
    else:
        row['prevApSchDepCnt']=4.0
        
        
    if 0<=row1['prevpercent_delay'] and row1['prevpercent_delay']<=0.1:
        row['prevApPerDelay']=0.0
    elif 0.1<row1['prevpercent_delay'] and row1['prevpercent_delay']<=0.4:
        row['prevApPerDelay']=1.0
    elif 0.4<row1['prevpercent_delay'] and row1['prevpercent_delay']<=0.7:
        row['prevApPerDelay']=2.0
    else:
        row['prevApPerDelay']=3.0
        
    if 0<=row1['prevDest_per_carr_count'] and row1['prevDest_per_carr_count']<=5:
        row['prevCarSchDepCnt']=0.0
    elif 5<row1['prevDest_per_carr_count'] and row1['prevDest_per_carr_count']<=10:
        row['prevCarSchDepCnt']=1.0
    elif 10<row1['prevDest_per_carr_count'] and row1['prevDest_per_carr_count']<=15:
        row['prevCarSchDepCnt']=2.0
    else:
        row['prevCarSchDepCnt']=3.0
        
        
    if 0<=row1['prevper_del_per_carr'] and row1['prevper_del_per_carr']<=0.1:
        row['prevCarPerDelay']=0.0
    elif 0.1<row1['prevper_del_per_carr'] and row1['prevper_del_per_carr']<=0.4:
        row['prevCarPerDelay']=1.0
    elif 0.4<row1['prevper_del_per_carr'] and row1['prevper_del_per_carr']<=0.7:
        row['prevCarPerDelay']=2.0
    else:
        row['prevCarPerDelay']=3.0
    
    
    
    Rows1[count]=row
        

In [21]:
discWithPrevDf = pd.DataFrame(Rows1)
discWithPrevDf = discWithPrevDf.T

In [81]:
#discWithPrevDf.UniqueCarrier.unique()
discWithPrevDf.head().T

,0,1,2,3,4
ActualArrTime,8.0,9.0,9.0,9.0,9.0
ActualDepTime,5.0,6.0,6.0,6.0,7.0
ApPerDelay,2.0,1.0,1.0,1.0,1.0
ApSchDepCnt,0.0,2.0,2.0,2.0,2.0
ArrDelay,0.0,0.0,1.0,1.0,0.0
ArrHr,8.0,9.0,8.0,9.0,8.0
CarPerDelay,0.0,0.0,1.0,1.0,1.0
CarSchDepCnt,0.0,0.0,1.0,1.0,1.0
DayOfWeek,0.0,0.0,0.0,0.0,0.0
DayofMonth,0.0,0.0,0.0,0.0,0.0


In [22]:
discWithPrevDf.to_csv(path_or_buf='./ORDDiscrete_15minBoundary_2008.csv')